# Perceptron no  lineal, de Widrow



## Big Picture

%%html
<img src= 'https://storage.googleapis.com/open-courses/imagenes-5c33/DeepLearningTimeline.jpg' />

Camino a Seguir


*   Perceptron de Rosenblatt
*   Perceptron de Widrow
*   Multiperceptron, backpropagation


## Objetivo desde el punto de vista Matemático

####Objetivo Primario

%%html
<img src= 'https://storage.googleapis.com/open-courses/imagenes-5c33/XOR.jpg' />

<p> Los perceptrones (Rosenblatt, Widrow) separan linealmente a puntos del espacio </p>

<p> Para separar puntos que NO son linealmente separables, necesito de varios perceptrones </p>

<p> Construire una Deep Neural Network de varios hidden layers y aprendere a modificar los pesos de cada layer propagando hacia atras los errores </p>

## Algoritmo  BackPropagation



*   Etapa forward,  avance normal de la red
*   Etapa backward, propago los errores hacia atras



### Etapa  Forward

%%html
<img src='https://storage.googleapis.com/open-courses/imagenes-5c33/forward01.jpg' />

%%html
<img src='https://storage.googleapis.com/open-courses/imagenes-5c33/forward02.jpg' />

%%html
<img src='https://storage.googleapis.com/open-courses/imagenes-5c33/forward03.jpg' />

## Etapa  Backward

%%html
<img src='https://storage.googleapis.com/open-courses/imagenes-5c33/back01.jpg' />

%%html
<img src='https://storage.googleapis.com/open-courses/imagenes-5c33/back02.jpg' />

%%html
<img src='https://storage.googleapis.com/open-courses/imagenes-5c33/back04.jpg' />

%%html
<img src='https://storage.googleapis.com/open-courses/imagenes-5c33/back03.jpg' />

### Resumen

%%html
<img src='https://storage.googleapis.com/open-courses/imagenes-5c33/resumen01.jpg' />

%%html
<img src='https://storage.googleapis.com/open-courses/imagenes-5c33/resumen02.jpg' />

%%html
<img src='https://storage.googleapis.com/open-courses/imagenes-5c33/resumen03.jpg' />

%%html
<img src='https://storage.googleapis.com/open-courses/imagenes-5c33/resumen04.jpg' />

### Código en Python del Algoritmo de BackPropagation

Código en Python que grafica como van evolucionando las rectas que separan a los positivos de los negativos

In [ ]:
# conexion al Google Drive
from google.colab import drive
drive.mount('/content/.drive')
!mkdir -p "/content/.drive/My Drive/DMA"
!mkdir -p "/content/buckets"
!ln -s "/content/.drive/My Drive/DMA" /content/buckets/b1

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display
import time
from sklearn.preprocessing import StandardScaler

In [ ]:
# definicion de las funciones de activacion
#  y sus derivadas
#  ahora agregando las versiones VECTORIZADAS

def func_eval(fname, x):
    match fname:
        case "purelin":
            y = x
        case "logsig":
            y = 1.0 / ( 1.0 + math.exp(-x) )
        case "tansig":
            y = 2.0 / ( 1.0 + math.exp(-2.0*x) ) - 1.0
    return y

# version vectorizada de func_eval
func_eval_vec = np.vectorize(func_eval)


def deriv_eval(fname, y):  #atencion que y es la entrada y=f( x )
    match fname:
        case "purelin":
            d = 1.0
        case "logsig":
            d = y*(1.0-y)
        case "tansig":
            d = 1.0 - y*y
    return d


# version vectorizada de deriv_eval
deriv_eval_vec = np.vectorize(deriv_eval)

In [ ]:
# definicion de la clase de graficos, NO hay x0 = 1

class perceptron_plot:
    def __init__(self, X, Y, delay) -> None:
        self.X = X
        self.Y = Y
        self.delay = delay
        x1_min = np.min(X[:,0])
        x2_min = np.min(X[:,1])
        x1_max = np.max(X[:,0])
        x2_max = np.max(X[:,1])
        self.x1_min = x1_min - 2.0*(x1_max - x1_min)
        self.x1_max = x1_max + 2.0*(x1_max - x1_min)
        self.x2_min = x2_min - 2.0*(x2_max - x2_min)
        self.x2_max = x2_max + 2.0*(x2_max - x2_min)
        self.fig = plt.figure(figsize = (10,8))
        self.ax = self.fig.subplots()
        self.ax.set_xlim(self.x1_min, self.x1_max, auto=False)
        self.ax.set_ylim(self.x2_min, self.x2_max, auto=False)

    def graficar(self, W, x0, epoch, fila) -> None:
        display.clear_output(wait =True)
        plt.cla()
        #self.ax = self.fig.subplots()

        self.ax.set_xlim(self.x1_min, self.x1_max)
        self.ax.set_ylim(self.x2_min, self.x2_max)
        plt.title( 'epoch ' + str(epoch) + '  reg ' + str(fila) )
        # ploteo puntos positivos
        self.ax.plot(self.X[self.Y==1,0], self.X[self.Y==1,1], 'o', color="green", markersize=10)
        # ploteo puntos negativos
        self.ax.plot(self.X[self.Y==0,0], self.X[self.Y==0,1], 'o', color="blue", markersize=10)

        # Sobreploteo el punto que no coincidio
        if(fila>=0):
            self.ax.plot(self.X[fila,0], self.X[fila,1], 'o',
                         color= ('green' if self.Y[fila]==1 else 'blue'),
                         markersize= 12, markeredgecolor= 'red')

        #dibujo le recta
        vx2_min = -(W[0]*self.x1_min + x0)/W[1]
        vx2_max = -(W[0]*self.x1_max + x0)/W[1]

        self.ax.plot([self.x1_min, self.x1_max],
                     [vx2_min, vx2_max],
                     linewidth = 2,
                     color = 'red',
                     alpha = 0.5)

        display.display(plt.gcf())
        #plt.cla()
        time.sleep(self.delay)


    def graficarVarias(self, W, x0, epoch, fila) -> None:
        display.clear_output(wait =True)
        plt.cla()
        #self.ax = self.fig.subplots()

        self.ax.set_xlim(self.x1_min, self.x1_max)
        self.ax.set_ylim(self.x2_min, self.x2_max)
        plt.title( 'epoch ' + str(epoch) + '  reg ' + str(fila))
        # ploteo puntos positivos
        self.ax.plot(self.X[self.Y==1,0], self.X[self.Y==1,1], 'o', color="green", markersize=10)
        # ploteo puntos negativos
        self.ax.plot(self.X[self.Y==-1,0], self.X[self.Y==-1,1], 'o', color="blue", markersize=10)
        self.ax.plot(self.X[self.Y==0,0], self.X[self.Y==0,1], 'o', color="blue", markersize=10)

        # Sobreploteo el punto que no coincidio
        if(fila>=0):
            self.ax.plot(self.X[fila,0], self.X[fila,1], 'o',
                         color= ('green' if self.Y[fila]==1 else 'blue'),
                         markersize= 12, markeredgecolor= 'red')

        # dibujo las rectas
        for i in range(len(x0)):
            #vx2_min = -(W[0,i]*self.x1_min + x0[i])/W[1,i]
            #vx2_max = -(W[0,i]*self.x1_max + x0[i])/W[1,i]
            vx2_min = -(W[i,0]*self.x1_min + x0[i])/W[i,1]
            vx2_max = -(W[i,0]*self.x1_max + x0[i])/W[i,1]

            self.ax.plot([self.x1_min, self.x1_max],
                         [vx2_min, vx2_max],
                         linewidth = 2,
                         color = 'red',
                         alpha = 0.5)

        display.display(plt.gcf())
        #plt.cla()
        time.sleep(self.delay)


In [ ]:
# Los datos que debe modelar la  Deep Neural Network
# El simple XOR,   de -1 a 1

registros = [[-1, -1], [-1, 1], [1, -1],[1, 1]]
clases = [-1, 1, 1, -1]

# Paso las listas a numpy
X = np.array(registros)
Y = np.array(clases).reshape(len(clases),1) # vector columna

# con estos puntos NO hace falta estandarizar

In [ ]:
# Arquitectura de la red
# Tamano datos

X_row = X.shape[0]
X_col = X.shape[1]

Y_col = 1

In [ ]:
# Arquitectura de la red
# Input Layer - Capa de entrada

input_size = X_col  # Tantas neuronas de entrada como campos de la tabla

In [ ]:
# Arquitectura de la red
# 1st  Hidden Layer

hidden01_size = 2  # neuronas de la  Primera capa oculta
hidden01_FUNC = 'logsig'  # uso la logistica  [0, 1]

In [ ]:
# Arquitectura de la red
# 2nd  Hidden Layer

# no va nada, NO hay en este caso segunda capa oculta

In [ ]:
# Arquitectura de la red
# 3rd  Hidden Layer

# no va nada, NO hay en este caso tercera capa oculta

In [ ]:
# Arquitectura de la red
# Output Layer - Capa de salida

output_size = Y_col  # Tantas neuronas de salida como campos de Y (uno solo)
output_FUNC = 'tansig'  # uso la tangente hiperbolica [-1, 1]

In [ ]:
# seteo de la semilla aleatoria
np.random.seed(102191) # mi querida random seed para que las corridas sean reproducibles

In [ ]:
# Incializo las matrices de pesos azarosamente

# W01 es la matriz de pesos que va del input a la Primera capa oculta
W01 = np.random.uniform(-0.5, 0.5, [hidden01_size, input_size])
X01 = np.random.uniform(-0.5, 0.5, [hidden01_size, 1] )

# W02 es la matriz de pesos que vn de la primera capa oculta a la capa de salida
W02 = np.random.uniform(-0.5, 0.5, [output_size, hidden01_size])
X02 = np.random.uniform(-0.5, 0.5, [output_size, 1] )

In [ ]:
# controles del entrenamiento

# Limite de lo que estoy dispuesto a trabajar
epoch_limit = 5000    # para terminar si no converge

# cuando la mejora del error sea inferior a este valor, me detendré
error_delta_umbral = 1.0e-07

# controla la velocidad de convergencia
learning_rate = 0.2

# controla cada cuanto se grafica un epoch
demora_impresion = 0.1  # segundos

In [ ]:
# inicializaciones del bucle

epoch = 0
error_epoch = float('inf')
error_ant =  0.0

In [ ]:
# el bucle principal del algoritmo BackPropagation

grafico = perceptron_plot(X=X, Y=Y.T[0], delay=demora_impresion)

# continuo mientras en la iteración anterior modifique algo  y NO llegué al límite de epochs
while ((math.fabs(error_epoch - error_ant) > error_delta_umbral) and (epoch < epoch_limit)):
    epoch += 1
    error_suma = 0.0
    error_ant = error_epoch

    # recorro siempre TODOS los registros de entrada
    for fila in range(X_row):
        # fila es el registro actual
        x = X[fila:fila+1,:] # ej  array([[-1, -1]])
        clase = Y[fila]

        # etapa Forward
        # propagar el input x hacia adelante
        hidden01_estimulos = W01 @ x.T + X01
        hidden01_salidas = func_eval_vec(hidden01_FUNC, hidden01_estimulos)

        output_estimulos = W02 @ hidden01_salidas + X02
        output_salidas = func_eval_vec(output_FUNC, output_estimulos)

        # etapa Backward
        # calculo los errores en la capa hidden y la capa output
        ErrorSalida = clase.T - output_salidas
        # output_delta es un solo numero
        output_delta = ErrorSalida * deriv_eval_vec(output_FUNC, output_salidas)
        # hidden_delta es un vector columna
        hidden_delta = deriv_eval_vec(hidden01_FUNC, hidden01_salidas)*(W02.T @ output_delta)

        # ya tengo los errores que comete cada capa
        # corregir matrices de pesos, voy hacia atras
        # backpropagation
        W01 = W01 + learning_rate * (hidden_delta @ x )
        X01 = X01 + learning_rate * hidden_delta
        W02 = W02 + learning_rate * (output_delta @ hidden01_salidas.T)
        X02 = X02 + learning_rate * output_delta


    # ya recalcule las matrices de pesos
    # ahora avanzo la red, feed-forward
    # para calcular el red(X) = Y
    hidden01_estimulos = W01 @ X.T + X01
    hidden01_salidas = func_eval_vec(hidden01_FUNC, hidden01_estimulos)

    output_estimulos = W02 @ hidden01_salidas + X02
    output_salidas = func_eval_vec(output_FUNC, output_estimulos)


    # calculo el error promedio general de TODOS los X
    error_epoch= np.mean( (Y.T - output_salidas)**2 )

    # Grafico las rectas de la Primer Hidden Layer
    # tengo que hacer X01.T[0]  para que pase el vector
    if( epoch % 10 == 0 ) : grafico.graficarVarias(W01, X01.T[0], epoch, -1)



In [ ]:
# el error
print("error_epoch= ", error_epoch)
print("error_ant = ", error_ant)
print("delta = ", math.fabs(error_epoch - error_ant))

In [ ]:
# la cantidad de epochs necesarias hasta encontrar una solucion
print("Para converger hicieron falta epochs=",epoch)

In [ ]:
# la matriz  W01 y X01, de la capa de entrada al primer Hidden Layer
print( "W01 = ", W01)
print( "X01 = ", X01)

In [ ]:
# la matriz  W02 y X02, del primer Hidden Layer a la capa de salida
print( "W02 = ", W02)
print( "X02 = ", X02)

In [ ]:
# calculo la salida de la red
#  comprouebo que NO son valores 0 o 1
#  lo que implica que deberé decidir mediante un umbral

print( "fila\tclase\testimulo\ty=f(estimulo)")
for fila in range(X_row):
    # fila es el registro actual
    x = X[fila:fila+1,:]
    clase = Y[fila]

    # propagar el x hacia adelante
    hidden01_estimulos = W01 @ x.T + X01
    hidden01_salidas = func_eval_vec(hidden01_FUNC, hidden01_estimulos)

    output_estimulos = W02 @ hidden01_salidas + X02
    output_salidas = func_eval_vec(output_FUNC, output_estimulos)


    # funcion de activacion
    print( fila, "\t", clase, "\t", output_estimulos, "\t", output_salidas)